# NH3 Historical Data

## 1. Import needed libraries and do other setup
- `pandas` for interpreting tabular data
- `numpy` for numerical operations
- a function from the `scipy` library that finds maxima and minima
- `datetime`, which interprets date/time data
- `matplotlib`, for graphing
- `ipywidgets`, for interactive widgets
- `BytesIO`, to read the uploaded file

In [ ]:
from __future__ import print_function

import pandas as pd
import numpy as np
from scipy.signal import find_peaks
import datetime as dt
from io import BytesIO

%matplotlib inline
from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = [16,9]

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Formatter to display tables side-by-side: https://stackoverflow.com/a/57832026/11639533
from IPython.display import display, HTML

def display_side_by_side(dfs:list, captions:list):
    """Display tables side by side to save vertical space
    Input:
        dfs: list of pandas.DataFrame
        captions: list of table captions
    """
    output = ""
    combined = dict(zip(captions, dfs))
    for caption, df in combined.items():
        output += df.style.set_table_attributes("style='display:inline'").set_caption(caption)._repr_html_()
        output += "\xa0\xa0\xa0"
    display(HTML(output))

# Prevent output cells from scrolling
from IPython.display import Javascript

def run_all(ev):
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
    
def reset(ev):
    display(Javascript('IPython.notebook.kernel.restart(); setTimeout(function(){ IPython.notebook.execute_all_cells(); }, 1000);'))

continue_button = widgets.Button(description="Continue",button_style='success',icon="arrow-right")
continue_button.on_click(run_all)

reset_button = widgets.Button(description="Reset")
reset_button.on_click(reset)

# Download link generator to export data: https://stackoverflow.com/a/42907645/11639533
import base64

def create_download_link(df, title = "Download CSV file", filename = "data.csv"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

## 2. Upload CSV file

In [ ]:
filepicker = widgets.FileUpload(
    accept='text/csv',
    button_style='info'
)
print("1. Click the upload button and select the file you want to analyze. The most recently uploaded file will be used.")
display(filepicker)
print("2. Hit continue to move to the next step in the analysis.")
display(continue_button)

## 3. Parse data from file

In [ ]:
try:
    raw = pd.read_csv(
        BytesIO(filepicker.data[0]),  # Read from uploaded file
        header=0,  # First row is a header
        #index_col=0,  # Use first column for row indices
        parse_dates=[0],  # Interpret first column as dates
    ).dropna()  # Delete null values
    print("This is the uploaded file. ")
    display(raw)
    
    timecol = widgets.Dropdown(options=raw.columns, description='Time:')
    if 'NH3' in raw.columns:
        nh3col = widgets.Dropdown(options=raw.columns, description='NH3:', value='NH3')
    else:
        nh3col = widgets.Dropdown(options=raw.columns, description='NH3:')
    print("Select the time and NH3 columns from the loaded file using the dropdown boxes below. Then click the continue button.")
    display(timecol, nh3col, continue_button)
except IndexError:
    print("Select a file using the upload button in Step 2, then click continue.")

In [ ]:
# Rename the columns to 'Time' and 'NH3' based on the selection above
try:
    df = raw[[timecol.value, nh3col.value]]
    df = df.rename(columns={timecol.value: "Time", nh3col.value: "NH3"})
except NameError:
    print("Select a file using the upload button in Step 2, then click continue.")

## 4. Set threshold values
We will find the maxima and minima using the `scipy.signal.find_peaks` function. To do so, a minimum threshold needs to be set for finding maximum values, as does a maximum for the minimums found.

In [ ]:
print("Adjust the cutoff sliders below so that most of the maximums are above the orange line, and most of the minimums")
print("are below the green one. If you want to type in a value, click the number to the right of the slider.")

def set_cutoff(orange=4.0, green=2.5):
    global max_cutoff, min_cutoff
    max_cutoff = orange
    min_cutoff = green
    plt.rcParams["figure.figsize"] = [16,9]
    plt.plot(df['Time'], df['NH3'])
    plt.axhline(y=max_cutoff, color='orange', linestyle='-', linewidth=3)
    plt.axhline(y=min_cutoff, color='green', linestyle='-', linewidth=3)
    plt.show()

cutoff_slider=interact(set_cutoff,
    orange=widgets.FloatSlider(value=4.0, min=0, max=8, step=0.01, continuous_update=False, layout=widgets.Layout(width='100%', color='orange')),
    green=widgets.FloatSlider(value=2.5, min=0, max=8, step=0.01, continuous_update=False, layout=widgets.Layout(width='100%')));

display(continue_button)


## 5. Detect peaks

In [ ]:
maxima, _ = find_peaks(df['NH3'], height=max_cutoff, prominence=0.3)
minima, _ = find_peaks(-df['NH3'], height=-min_cutoff, prominence=0.3)

maxima = df.iloc[maxima]
minima = df.iloc[minima]

with pd.option_context('mode.chained_assignment', None):
    maxima.loc[:, 'type']='max'
    minima.loc[:, 'type']='min'

print('Here are the peaks detected. You can change the thresholds above and hit "continue" to update this graph')
fig, ax = plt.subplots(figsize=(16, 9))
plt.plot(df['Time'], df['NH3'])
plt.plot(maxima['Time'], maxima['NH3'], "o")
plt.plot(minima['Time'], minima['NH3'], "o")
plt.axhline(y=max_cutoff, color='orange', linestyle='--', linewidth=2)
plt.axhline(y=min_cutoff, color='green', linestyle='--', linewidth=2)
plt.show()

## 6. Rate Analysis
Next, we find each interval of changing concentration based on these extrema. The rate of change is calculated for each and displayed on a graph below.

In [ ]:
cmax = None
cmin = None
extrema = pd.merge_ordered(maxima, minima)

# Make sure entries alternate between max and min, delete points that do not alternate
i = 0
imax=len(extrema)

while (i+1 < imax):
    if extrema['type'][i] == extrema['type'][i+1]:
        #print(extrema.iloc[i])
        extrema = extrema.drop(i, axis=0)
    i += 1

extrema['Delta_NH3'] = extrema['NH3'].diff()
extrema['Delta_Time'] = extrema['Time'].diff()
extrema = extrema.drop(0, axis=0)  # First row has no previous value to divide, thus has NaT time and NaN delta NH3
extrema['NH3/Hour'] = extrema['Delta_NH3']/[pd.Timedelta.total_seconds(t)/3600 for t in extrema['Delta_Time']]

display_side_by_side(
    [
        extrema[['Time', 'Delta_Time', 'NH3', 'Delta_NH3', 'NH3/Hour']][extrema['type'] == 'max'],
        extrema[['Time', 'Delta_Time', 'NH3', 'Delta_NH3', 'NH3/Hour']][extrema['type'] == 'min']
    ], ["Maxima", "Minima"])


**You can download these maximum and minimum values using the link below.**

In [ ]:
create_download_link(extrema)

## 6. Rate Visualization

In [ ]:
# Plot resulting rates

def plotrates(ymax=10, ymin=-10):
    fig, ax = plt.subplots(figsize=(16, 9))
    plt.rcParams["figure.figsize"] = [16,9]
    plt.ylim(top=ymax, bottom=ymin)
    increases = plt.plot(extrema['Time'][extrema['type'] == 'max'], extrema['NH3/Hour'][extrema['type'] == 'max'], color='green')
    decreases = plt.plot(extrema['Time'][extrema['type'] == 'min'], extrema['NH3/Hour'][extrema['type'] == 'min'], color='red')
    rateplot = plt.show()
print("You can adjust the scale on the y-axis using these sliders:")
interact(plotrates,
    ymax=widgets.FloatSlider(value=10, min=0, max=20, step=0.1, continuous_update=False, layout=widgets.Layout(width='100%')),
    ymin=widgets.FloatSlider(value=-10, min=-20, max=0, step=0.1, continuous_update=False, layout=widgets.Layout(width='100%')));